In [1]:
# packages needed (listed in requirenments file)

!pip install matplotlib numpy pandas seaborn scikit-learn torch tqdm transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Model without reference

In [15]:
# Works, dont edit!
# Retrain the 01_model_without_reference with the original model

"""
==============================
Model Training - No Reference
==============================
This script takes our cleaned twitter data and builds a pipeline to fine-tune a BERT which can classify the tweet into its 3 labels classes. 

The label classes are as follows: 
Type (3 labels): Antagonizing, Other political statement, Unclassifiable
Tone Hateful (3 labels):  Hateful, Not Hateful, Unclassifiable Hateful 
Tone Constructive (3 labels): Constructive, Not Constructive, Unclassifiable Constructive

"""

# Data processing 
import pandas as pd
import numpy as np
import argparse
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns


# Modelling 
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader


def main():
    """
    ==================
    ----- Set Up -----
    ==================
    """
    # -- Parameters -- 
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Change model here if desired
    batch_size = 16 # Small batch number for processing, can increase if using GPU
    num_labels = 12

    # Load Data 
    train_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/train/train_data_no_ref.csv").head(864) #smaller sample for testing
    test_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/test/test_data_no_ref.csv").head(216)

    # -- Classes -- 
    class TextDataset(Dataset):
        # initialise the text, labels, the chosen tokenizer (BERT) and set the maximum length to BERT's max (512)
        def __init__(self, texts, labels, tokenizer, max_length=512):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length

        # Return the total number of instances in the dataset (i.e., the number of rows)
        def __len__(self):
            return len(self.texts)

        # Take the text and labels as indexes, and return a dictonary of the tokenised text and its corresponding label 
        def __getitem__(self, idx):
            text = self.texts[idx]
            label = self.labels[idx]
            encodings = self.tokenizer.encode_plus(
                text,
                padding="max_length",
                truncation=True,
                max_length=self.max_length,
                return_tensors="pt"
            )
            return {"input_ids": encodings["input_ids"][0], "attention_mask": encodings["attention_mask"][0], "labels": torch.tensor(label, dtype=torch.float)}

    """
    ==================
    ----- Model -----
    ==================
    """
    # -- Define labels -- 
    mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])
                                   
    train_labels = mlb.fit_transform(train_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)
    test_labels = mlb.transform(test_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)

    # -- Create datasets -- 
    train_dataset = TextDataset(train_data['rep_text'].tolist(), train_labels, tokenizer, max_length=512)
    test_dataset = TextDataset(test_data['rep_text'].tolist(), test_labels, tokenizer, max_length=512)

    # create train and test dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    # -- Set the optimizer, loss function and learning rate -- 
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=2e-5)

    # -- Training model -- 
    def train(model, dataloader, criterion, optimizer, device):
        model.train()
        model.to(device)
        total_loss = 0
        
        for batch in tqdm(dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    # -- Evaluate Function -- 
    def evaluate(model, dataloader, criterion, device):
        model.eval()
        model.to(device)
        total_loss = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Evaluating"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                loss = criterion(logits, labels)
                total_loss += loss.item()

                preds = (logits.sigmoid() > 0.5).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

        avg_loss = total_loss / len(dataloader)

        return avg_loss, np.array(all_labels), np.array(all_preds)

    """
    ======================
    ----- Test model -----
    ======================
    """
    #  -- Call device --
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # -- Setup saving parameters -- 
    num_epochs = 1
    output_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_without_reference/"
    epoch_no = 0 

    # Define a path to save the model checkpoints
    checkpoint_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_without_reference/checkpoints/"

    # Define the model checkpoint file format (e.g., "model_epoch_{epoch}.pt")
    checkpoint_filename = "model_epoch_{epoch}.pt"

    # Define the label classes
    label_classes = {
        'rep_type': ['Antagonizing', 'Other political statement', 'Unclassifiable'],
        'rep_constructive': ['Constructive', 'Not Constructive', 'Unclassifiable Constructive'],
        'rep_hateful': ['Hateful', 'Not Hateful', 'Unclassifiable Hateful'],
        'rep_agree': ['Agree', 'Disagree', 'Unclear']
    }

    for epoch in range(num_epochs):
        # Free up memory by deleting variables
        epoch_no = epoch_no + 1
        print(f"Epoch {epoch_no}:")

        # -- Train model -- 
        train_loss = train(model, train_dataloader, criterion, optimizer, device)
        print(f"Train Loss: {train_loss}")

        # Save model checkpoint
        checkpoint_path = checkpoint_dir + checkpoint_filename.format(epoch=epoch_no)
        torch.save(model.state_dict(), checkpoint_path)

        # -- Test model -- 
        test_loss, test_true_labels, test_pred_labels = evaluate(model, test_dataloader, criterion, device)
        print(f"Test Loss: {test_loss}")

        # Convert output from one hot encoding 
        true_labels = np.array(test_true_labels)
        pred_labels = np.array(test_pred_labels)

        # Generate confusion matrix
        for class_name, labels in label_classes.items():
            # Get indices corresponding to labels in this class
            indices = [i for i, label in enumerate(mlb.classes_) if label in labels]
            
            # Extract true and predicted values for this class
            class_true = np.argmax(true_labels[:, indices], axis=1)
            class_pred = np.argmax(pred_labels[:, indices], axis=1)
            
            # Compute the confusion matrix
            matrix = confusion_matrix(class_true, class_pred)

            # Save or visualize the confusion matrix
            plt.figure(figsize=(10,10))
            sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels,
                    yticklabels=labels)
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.title(f'Confusion Matrix for {class_name}')
            plt.savefig(f'{output_dir}confusion_matrix_{class_name}.png')
            plt.close()

        # Generate report 
        report = classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_, output_dict=True)
        report_name = output_dir + "classification_report_" + str(epoch_no) + ".txt"

        with open(report_name, "w") as f:
            f.write(classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_))
        
        # Print results 
        #print(f"Epoch {epoch+1}:")
        print(f"  Train Loss = {train_loss:.4f}")
        print(f"  Test Loss = {test_loss:.4f}")

Note to self: there are 5184 texts in the training data: train_data_no_ref.csv, and depeding on the batch size, the number of batches pr epoch (number displayed in the tqdm when training the model) will change (e.g. 16 batches will be 324 because 5184/16= 324)


In [16]:
main()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will b

Epoch 1:


Training: 100%|█████████████████████████████████| 54/54 [18:40<00:00, 20.75s/it]


Train Loss: 0.5637614219276993


Evaluating: 100%|███████████████████████████████| 14/14 [00:43<00:00,  3.08s/it]


Test Loss: 0.4918421357870102
  Train Loss = 0.5638
  Test Loss = 0.4918


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
# Retrain the 01_model_without_reference with a new model (XLM-Roberta)
# looks to be working, need to fully run

"""
==============================
Model Training - No Reference
==============================
This script takes our cleaned twitter data and builds a pipeline to fine-tune a BERT which can classify the tweet into its 3 labels classes. 

The label classes are as follows: 
Type (3 labels): Antagonizing, Other political statement, Unclassifiable
Tone Hateful (3 labels):  Hateful, Not Hateful, Unclassifiable Hateful 
Tone Constructive (3 labels): Constructive, Not Constructive, Unclassifiable Constructive

"""

# Data processing 
import pandas as pd
import numpy as np
import argparse
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns


# Modelling 
from transformers import BertTokenizer, BertForSequenceClassification

#new
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader


def main_2():
    """
    ==================
    ----- Set Up -----
    ==================
    """
    # -- Parameters -- 
    tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large") # Change model here if desired
    batch_size = 16 # Small batch number for processing, can increase if using GPU
    num_labels = 12

    # Load Data 
    train_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/train/train_data_no_ref.csv").head(864) #smaller sample for testing
    test_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/test/test_data_no_ref.csv").head(216)

    # -- Classes -- 
    class TextDataset(Dataset):
        # initialise the text, labels, the chosen tokenizer (BERT) and set the maximum length to BERT's max (512)
        def __init__(self, texts, labels, tokenizer, max_length=512):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length

        # Return the total number of instances in the dataset (i.e., the number of rows)
        def __len__(self):
            return len(self.texts)

        # Take the text and labels as indexes, and return a dictonary of the tokenised text and its corresponding label 
        def __getitem__(self, idx):
            text = self.texts[idx]
            label = self.labels[idx]
            encodings = self.tokenizer.encode_plus(
                text,
                padding="max_length",
                truncation=True,
                max_length=self.max_length,
                return_tensors="pt"
            )
            return {"input_ids": encodings["input_ids"][0], "attention_mask": encodings["attention_mask"][0], "labels": torch.tensor(label, dtype=torch.float)}

    """
    ==================
    ----- Model -----
    ==================
    """
    # -- Define labels -- 
    mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])
                                   
    train_labels = mlb.fit_transform(train_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)
    test_labels = mlb.transform(test_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)

    # -- Create datasets -- 
    train_dataset = TextDataset(train_data['rep_text'].tolist(), train_labels, tokenizer, max_length=512)
    test_dataset = TextDataset(test_data['rep_text'].tolist(), test_labels, tokenizer, max_length=512)

    # create train and test dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    # -- Set the optimizer, loss function and learning rate -- 
    model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-large", num_labels=num_labels)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=2e-5)

    # -- Training model -- 
    def train(model, dataloader, criterion, optimizer, device):
        model.train()
        model.to(device)
        total_loss = 0
        
        for batch in tqdm(dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    # -- Evaluate Function -- 
    def evaluate(model, dataloader, criterion, device):
        model.eval()
        model.to(device)
        total_loss = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Evaluating"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                loss = criterion(logits, labels)
                total_loss += loss.item()

                preds = (logits.sigmoid() > 0.5).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

        avg_loss = total_loss / len(dataloader)

        return avg_loss, np.array(all_labels), np.array(all_preds)

    """
    ======================
    ----- Test model -----
    ======================
    """
    #  -- Call device --
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # -- Setup saving parameters -- 
    num_epochs = 1
    output_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_without_reference_new/"
    epoch_no = 0 

    # Define a path to save the model checkpoints
    checkpoint_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_without_reference_new/checkpoints/"

    # Define the model checkpoint file format (e.g., "model_epoch_{epoch}.pt")
    checkpoint_filename = "model_epoch_{epoch}.pt"

    # Define the label classes
    label_classes = {
        'rep_type': ['Antagonizing', 'Other political statement', 'Unclassifiable'],
        'rep_constructive': ['Constructive', 'Not Constructive', 'Unclassifiable Constructive'],
        'rep_hateful': ['Hateful', 'Not Hateful', 'Unclassifiable Hateful'],
        'rep_agree': ['Agree', 'Disagree', 'Unclear']
    }

    for epoch in range(num_epochs):
        # Free up memory by deleting variables
        epoch_no = epoch_no + 1
        print(f"Epoch {epoch_no}:")

        # -- Train model -- 
        train_loss = train(model, train_dataloader, criterion, optimizer, device)
        print(f"Train Loss: {train_loss}")

        # Save model checkpoint
        checkpoint_path = checkpoint_dir + checkpoint_filename.format(epoch=epoch_no)
        torch.save(model.state_dict(), checkpoint_path)

        # -- Test model -- 
        test_loss, test_true_labels, test_pred_labels = evaluate(model, test_dataloader, criterion, device)
        print(f"Test Loss: {test_loss}")

        # Convert output from one hot encoding 
        true_labels = np.array(test_true_labels)
        pred_labels = np.array(test_pred_labels)

        # Generate confusion matrix
        for class_name, labels in label_classes.items():
            # Get indices corresponding to labels in this class
            indices = [i for i, label in enumerate(mlb.classes_) if label in labels]
            
            # Extract true and predicted values for this class
            class_true = np.argmax(true_labels[:, indices], axis=1)
            class_pred = np.argmax(pred_labels[:, indices], axis=1)
            
            # Compute the confusion matrix
            matrix = confusion_matrix(class_true, class_pred)

            # Save or visualize the confusion matrix
            plt.figure(figsize=(10,10))
            sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels,
                    yticklabels=labels)
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.title(f'Confusion Matrix for {class_name}')
            plt.savefig(f'{output_dir}confusion_matrix_{class_name}.png')
            plt.close()

        # Generate report 
        report = classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_, output_dict=True)
        report_name = output_dir + "classification_report_" + str(epoch_no) + ".txt"

        with open(report_name, "w") as f:
            f.write(classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_))
        
        # Print results 
        #print(f"Epoch {epoch+1}:")
        print(f"  Train Loss = {train_loss:.4f}")
        print(f"  Test Loss = {test_loss:.4f}")

In [7]:
main_2()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1:


Training:  26%|███████▊                      | 14/54 [25:07<1:11:46, 107.66s/it]


KeyboardInterrupt: 

# Model with reference

In [8]:
# Works, dont edit
# Retrain the 02_model_with_ref with the original model

"""
=================================
Model Training 2 - Reference Text
=================================

This script takes our cleaned twitter data and builds a pipeline to fine-tune a BERT which can classify the tweet into its 3 labels classes. 

The label classes are as follows: 
Type (3 labels): Antagonizing, Other political statement, Unclassifiable
Tone Hateful (3 labels):  Hateful, Not Hateful, Unclassifiable Hateful 
Tone Constructive (3 labels): Constructive, Not Constructive, Unclassifiable Constructive

To include the reference tweet information, there is one modification to this model from the basic 'No Reference Model' (01_model_no_ref.py): 
1. The reference text is appended to the reply tweet text with a [SEP] token to delineate the two tweets before the classification is made.
    - This means the model has more information to base its prediction upon as it can see patterns in the reference text and how this may influence the reply tweet text. 

Usage:
  $ python3 src/02_model_with_ref.py

"""

"""
=================================
----- Import Depenendencies -----
=================================
"""
# Data processing 
import pandas as pd
import numpy as np
import argparse
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling 
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertModel, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

"""
=========================
----- Main Function -----
=========================
"""
def main_3():
    """
    ==========================
    Parameters and Directories 
    ==========================
    """
    # -- Model parameters -- 
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Can change BERT model
    batch_size = 16     # small batch size due to computation, increase to 32 if using GPU
    num_labels = 12
    num_epochs = 1      # Increase if desired, model started to overfit after around 5 epochs
    epoch_no = 0 

    # -- Directories --
    #input_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data"
    output_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_with_reference_text/"
    checkpoint_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_with_reference_text/checkpoints/" # path for checkpoints to be saved


    """
    =======
    Classes
    =======
    """
    # create a Dataset class for text and reference labels
    class TextDatasetWithRefLabels(Dataset):
        def __init__(self, ref_texts, ref_labels, texts, labels, tokenizer, max_length):
            self.ref_texts = ref_texts
            self.ref_labels = ref_labels
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length

            # Binarize the reference labels
            self.mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])
            self.ref_labels = self.mlb.fit_transform(ref_labels)

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, index):
            ref_text = self.ref_texts[index]
            ref_label = self.ref_labels[index]
            text = self.texts[index]
            label = self.labels[index]

            # Tokenize the reply tweet 
            rep_tokens = self.tokenizer.tokenize(text)

            # Determine the maximum length of the reference text that can be used 
            max_ref_length = 511 - len(rep_tokens)

            # Tokenize the reference text and truncate if necessary
            ref_tokens = self.tokenizer.tokenize(ref_text)
            if len(ref_tokens) > max_ref_length:
                truncated_ref_tokens = ref_tokens[-max_ref_length:]
            else:
                truncated_ref_tokens = ref_tokens
            
            # Combine the reference and reply text tokens with a [SEP] token
            combined_tokens = truncated_ref_tokens + [self.tokenizer.sep_token] + rep_tokens 

            # Convert the combined tokens to a text string
            combined_text = self.tokenizer.convert_tokens_to_string(combined_tokens)

            # Encode the combined text
            encoded_data = self.tokenizer.encode_plus(
                combined_text,
                add_special_tokens=True,
                max_length=512,
                padding='max_length',
                truncation='only_first',
                return_attention_mask=True,
                return_tensors='pt'
            )

            input_ids = encoded_data['input_ids']
            attention_mask = encoded_data['attention_mask']

            # Convert the label data to tensors
            label_tensor = torch.tensor(label, dtype=torch.float32)
            ref_label_tensor = torch.tensor(ref_label, dtype=torch.float32)

            return {
                'input_ids': input_ids.squeeze(0),
                'attention_mask': attention_mask.squeeze(0),
                'ref_labels': ref_label_tensor,
                'labels': label_tensor
            }

    class BertForContextualClassification(nn.Module):
        def __init__(self, num_labels):
            super(BertForContextualClassification, self).__init__()
            self.bert = BertModel.from_pretrained('bert-base-uncased')
            self.dropout = nn.Dropout(0.1)
            self.linear = nn.Linear(self.bert.config.hidden_size, num_labels)

        def forward(self, input_ids, attention_mask, ref_input_ids=None, ref_attention_mask=None):
            if ref_input_ids is not None:
                output = self.bert(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    inputs_embeds=None
                )
            else:
                output = self.bert(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    inputs_embeds=None
                )
            pooled_output = output[1]
            pooled_output = self.dropout(pooled_output)
            logits = self.linear(pooled_output)
            return logits

    """
    =============
    Preprocessing
    =============
    """
    # -- Load Data -- 
    train_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/train/train_data_ref.csv").head(864) #smaller sample for testing
    test_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/test/test_data_ref.csv").head(216)

    # prepare the train and test labels using MultiLabelBinarizer
    mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])

    train_labels = mlb.fit_transform(train_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)
    test_labels = mlb.transform(test_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)

    # -- create train and test datasets -- 
    train_dataset = TextDatasetWithRefLabels(train_data['ref_text'].tolist(),
                                            train_data[['ref_type', 'ref_hateful', 'ref_constructive']].values,
                                            train_data['rep_text'].tolist(),
                                            train_labels,
                                            tokenizer,
                                            max_length=512)

    test_dataset = TextDatasetWithRefLabels(test_data['ref_text'].tolist(),
                                            test_data[['ref_type', 'ref_hateful', 'ref_constructive']].values,
                                            test_data['rep_text'].tolist(),
                                            test_labels,
                                            tokenizer,
                                            max_length=512)
    
    # create train and test dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    """
    =====
    Model
    =====
    """
    # Initialize the model
    model = BertForContextualClassification(num_labels)

    # Define your optimizer, loss function and learning rate
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    criterion = nn.BCEWithLogitsLoss()

    # Create training function 
    def train(model, dataloader, criterion, optimizer, device):
        model.train()
        model.to(device)
        total_loss = 0

        for batch in tqdm(dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            ref_labels = batch["ref_labels"].to(device)
            #print("input_ids shape:", input_ids.shape)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask=attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    # -- create evaluation function -- 
    def evaluate(model, dataloader, criterion, device):
        model.eval()
        model.to(device)
        total_loss = 0
        all_preds = []
        all_labels = []

        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, attention_mask=attention_mask)

            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = (logits.sigmoid() > 0.5).cpu().detach().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().detach().numpy())

        avg_loss = total_loss / len(dataloader)

        return avg_loss, np.array(all_labels), np.array(all_preds)
    

    # Define the model checkpoint file format (e.g., "model_epoch_{epoch}.pt")
    checkpoint_filename = "model_epoch_{epoch}.pt"

    # Define the label classes
    label_classes = {
        'rep_type': ['Antagonizing', 'Other political statement', 'Unclassifiable'],
        'rep_constructive': ['Constructive', 'Not Constructive', 'Unclassifiable Constructive'],
        'rep_hateful': ['Hateful', 'Not Hateful', 'Unclassifiable Hateful'],
        'rep_agree': ['Agree', 'Disagree', 'Unclear']
    }
    
    # -- Wrap it up into a function -- 
    # Call device 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for epoch in range(num_epochs):
        # Free up memory by deleting variables
        epoch_no = epoch_no + 1
        print(f"Epoch {epoch_no}:")
        # -- Train model -- 
        train_loss = train(model, train_dataloader, criterion, optimizer, device)
        print(f"Train Loss: {train_loss}")

        # Save model checkpoint
        checkpoint_path = checkpoint_dir + checkpoint_filename.format(epoch=epoch_no)
        torch.save(model.state_dict(), checkpoint_path)

        # Evaluate the training data 
        #train_loss, true_labels, pred_labels = evaluate(model, train_dataloader, criterion, device)

        # -- Test model -- 
        test_loss, test_true_labels, test_pred_labels = evaluate(model, test_dataloader, criterion, device)
        print(f"Test Loss: {test_loss}")

        # Convert output from one hot encoding 
        true_labels = np.array(test_true_labels)
        pred_labels = np.array(test_pred_labels)

        # Generate confusion matrix
        for class_name, labels in label_classes.items():
            # Get indices corresponding to labels in this class
            indices = [i for i, label in enumerate(mlb.classes_) if label in labels]
            
            # Extract true and predicted values for this class
            class_true = np.argmax(true_labels[:, indices], axis=1)
            class_pred = np.argmax(pred_labels[:, indices], axis=1)
            
            # Compute the confusion matrix
            matrix = confusion_matrix(class_true, class_pred)

            # Save or visualize the confusion matrix
            plt.figure(figsize=(10,10))
            sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels,
                    yticklabels=labels)
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.title(f'Confusion Matrix for {class_name}')
            plt.savefig(f'{output_dir}confusion_matrix_{class_name}.png')
            plt.close()

        # Generate report 
        report = classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_, output_dict=True)
        report_name = output_dir + "classification_report_" + str(epoch_no) + ".txt"

        with open(report_name, "w") as f:
            f.write(classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_))
        
        # Print results 
        #print(f"Epoch {epoch+1}:")
        print(f"  Train Loss = {train_loss:.4f}")
        print(f"  Test Loss = {test_loss:.4f}")

        # Activate if kernel struggles to run model 
        #if epoch < 5:
            #del train_loss, true_labels, pred_labels, test_loss, test_true_labels, test_pred_labels


In [9]:
main_3()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will b

Epoch 1:


Training: 100%|█████████████████████████████████| 54/54 [18:46<00:00, 20.86s/it]


Train Loss: 0.548078970224769


Evaluating: 100%|███████████████████████████████| 14/14 [02:38<00:00, 11.31s/it]


Test Loss: 0.46460294297763277
  Train Loss = 0.5481
  Test Loss = 0.4646


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# Retrain the 02_model_with_ref with a new model (XLM-Roberta)
# looks to be working, need to fully run

"""
=================================
Model Training 2 - Reference Text
=================================

This script takes our cleaned twitter data and builds a pipeline to fine-tune a BERT which can classify the tweet into its 3 labels classes. 

The label classes are as follows: 
Type (3 labels): Antagonizing, Other political statement, Unclassifiable
Tone Hateful (3 labels):  Hateful, Not Hateful, Unclassifiable Hateful 
Tone Constructive (3 labels): Constructive, Not Constructive, Unclassifiable Constructive

To include the reference tweet information, there is one modification to this model from the basic 'No Reference Model' (01_model_no_ref.py): 
1. The reference text is appended to the reply tweet text with a [SEP] token to delineate the two tweets before the classification is made.
    - This means the model has more information to base its prediction upon as it can see patterns in the reference text and how this may influence the reply tweet text. 

Usage:
  $ python3 src/02_model_with_ref.py

"""

"""
=================================
----- Import Depenendencies -----
=================================
"""
# Data processing 
import pandas as pd
import numpy as np
import argparse
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling 
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# new
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import XLMRobertaModel

from transformers import BertModel, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

"""
=========================
----- Main Function -----
=========================
"""

def main_4():
    """
    ==========================
    Parameters and Directories 
    ==========================
    """
    # -- Model parameters -- 
    tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large") # Can change BERT model
    batch_size = 16     # small batch size due to computation, increase to 32 if using GPU
    num_labels = 12
    num_epochs = 1      # Increase if desired, model started to overfit after around 5 epochs
    epoch_no = 0 

    # -- Directories --
    #input_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data"
    output_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_with_reference_text_new/"
    checkpoint_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_with_reference_text_new/checkpoints/" # path for checkpoints to be saved


    """
    =======
    Classes
    =======
    """
    # create a Dataset class for text and reference labels
    class TextDatasetWithRefLabels(Dataset):
        def __init__(self, ref_texts, ref_labels, texts, labels, tokenizer, max_length):
            self.ref_texts = ref_texts
            self.ref_labels = ref_labels
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length

            # Binarize the reference labels
            self.mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])
            self.ref_labels = self.mlb.fit_transform(ref_labels)

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, index):
            ref_text = self.ref_texts[index]
            ref_label = self.ref_labels[index]
            text = self.texts[index]
            label = self.labels[index]

            # Tokenize the reply tweet 
            rep_tokens = self.tokenizer.tokenize(text)

            # Determine the maximum length of the reference text that can be used 
            max_ref_length = 511 - len(rep_tokens)

            # Tokenize the reference text and truncate if necessary
            ref_tokens = self.tokenizer.tokenize(ref_text)
            if len(ref_tokens) > max_ref_length:
                truncated_ref_tokens = ref_tokens[-max_ref_length:]
            else:
                truncated_ref_tokens = ref_tokens
            
            # Combine the reference and reply text tokens with a [SEP] token
            combined_tokens = truncated_ref_tokens + [self.tokenizer.sep_token] + rep_tokens 

            # Convert the combined tokens to a text string
            combined_text = self.tokenizer.convert_tokens_to_string(combined_tokens)

            # Encode the combined text
            encoded_data = self.tokenizer.encode_plus(
                combined_text,
                add_special_tokens=True,
                max_length=512,
                padding='max_length',
                truncation='only_first',
                return_attention_mask=True,
                return_tensors='pt'
            )

            input_ids = encoded_data['input_ids']
            attention_mask = encoded_data['attention_mask']

            # Convert the label data to tensors
            label_tensor = torch.tensor(label, dtype=torch.float32)
            ref_label_tensor = torch.tensor(ref_label, dtype=torch.float32)

            return {
                'input_ids': input_ids.squeeze(0),
                'attention_mask': attention_mask.squeeze(0),
                'ref_labels': ref_label_tensor,
                'labels': label_tensor
            }

    class BertForContextualClassification(nn.Module):
        def __init__(self, num_labels):
            super(BertForContextualClassification, self).__init__()
            self.bert = XLMRobertaModel.from_pretrained('xlm-roberta-large')
            self.dropout = nn.Dropout(0.1)
            self.linear = nn.Linear(self.bert.config.hidden_size, num_labels)

        def forward(self, input_ids, attention_mask, ref_input_ids=None, ref_attention_mask=None):
            if ref_input_ids is not None:
                output = self.bert(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    inputs_embeds=None
                )
            else:
                output = self.bert(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    inputs_embeds=None
                )
            pooled_output = output[1]
            pooled_output = self.dropout(pooled_output)
            logits = self.linear(pooled_output)
            return logits

    """
    =============
    Preprocessing
    =============
    """
    # -- Load Data -- 
    train_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/train/train_data_ref.csv").head(864) #smaller sample for testing
    test_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/test/test_data_ref.csv").head(216)

    # prepare the train and test labels using MultiLabelBinarizer
    mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])

    train_labels = mlb.fit_transform(train_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)
    test_labels = mlb.transform(test_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)

    # -- create train and test datasets -- 
    train_dataset = TextDatasetWithRefLabels(train_data['ref_text'].tolist(),
                                            train_data[['ref_type', 'ref_hateful', 'ref_constructive']].values,
                                            train_data['rep_text'].tolist(),
                                            train_labels,
                                            tokenizer,
                                            max_length=512)

    test_dataset = TextDatasetWithRefLabels(test_data['ref_text'].tolist(),
                                            test_data[['ref_type', 'ref_hateful', 'ref_constructive']].values,
                                            test_data['rep_text'].tolist(),
                                            test_labels,
                                            tokenizer,
                                            max_length=512)
    
    # create train and test dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    """
    =====
    Model
    =====
    """
    # Initialize the model
    model = BertForContextualClassification(num_labels)

    # Define your optimizer, loss function and learning rate
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    criterion = nn.BCEWithLogitsLoss()

    # Create training function 
    def train(model, dataloader, criterion, optimizer, device):
        model.train()
        model.to(device)
        total_loss = 0

        for batch in tqdm(dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            ref_labels = batch["ref_labels"].to(device)
            #print("input_ids shape:", input_ids.shape)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask=attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    # -- create evaluation function -- 
    def evaluate(model, dataloader, criterion, device):
        model.eval()
        model.to(device)
        total_loss = 0
        all_preds = []
        all_labels = []

        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, attention_mask=attention_mask)

            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = (logits.sigmoid() > 0.5).cpu().detach().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().detach().numpy())

        avg_loss = total_loss / len(dataloader)

        return avg_loss, np.array(all_labels), np.array(all_preds)
    

    # Define the model checkpoint file format (e.g., "model_epoch_{epoch}.pt")
    checkpoint_filename = "model_epoch_{epoch}.pt"

    # Define the label classes
    label_classes = {
        'rep_type': ['Antagonizing', 'Other political statement', 'Unclassifiable'],
        'rep_constructive': ['Constructive', 'Not Constructive', 'Unclassifiable Constructive'],
        'rep_hateful': ['Hateful', 'Not Hateful', 'Unclassifiable Hateful'],
        'rep_agree': ['Agree', 'Disagree', 'Unclear']
    }
    
    # -- Wrap it up into a function -- 
    # Call device 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for epoch in range(num_epochs):
        # Free up memory by deleting variables
        epoch_no = epoch_no + 1
        print(f"Epoch {epoch_no}:")
        # -- Train model -- 
        train_loss = train(model, train_dataloader, criterion, optimizer, device)
        print(f"Train Loss: {train_loss}")

        # Save model checkpoint
        checkpoint_path = checkpoint_dir + checkpoint_filename.format(epoch=epoch_no)
        torch.save(model.state_dict(), checkpoint_path)

        # Evaluate the training data 
        #train_loss, true_labels, pred_labels = evaluate(model, train_dataloader, criterion, device)

        # -- Test model -- 
        test_loss, test_true_labels, test_pred_labels = evaluate(model, test_dataloader, criterion, device)
        print(f"Test Loss: {test_loss}")

        # Convert output from one hot encoding 
        true_labels = np.array(test_true_labels)
        pred_labels = np.array(test_pred_labels)

        # Generate confusion matrix
        for class_name, labels in label_classes.items():
            # Get indices corresponding to labels in this class
            indices = [i for i, label in enumerate(mlb.classes_) if label in labels]
            
            # Extract true and predicted values for this class
            class_true = np.argmax(true_labels[:, indices], axis=1)
            class_pred = np.argmax(pred_labels[:, indices], axis=1)
            
            # Compute the confusion matrix
            matrix = confusion_matrix(class_true, class_pred)

            # Save or visualize the confusion matrix
            plt.figure(figsize=(10,10))
            sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels,
                    yticklabels=labels)
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.title(f'Confusion Matrix for {class_name}')
            plt.savefig(f'{output_dir}confusion_matrix_{class_name}.png')
            plt.close()

        # Generate report 
        report = classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_, output_dict=True)
        report_name = output_dir + "classification_report_" + str(epoch_no) + ".txt"

        with open(report_name, "w") as f:
            f.write(classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_))
        
        # Print results 
        #print(f"Epoch {epoch+1}:")
        print(f"  Train Loss = {train_loss:.4f}")
        print(f"  Test Loss = {test_loss:.4f}")

        # Activate if kernel struggles to run model 
        #if epoch < 5:
            #del train_loss, true_labels, pred_labels, test_loss, test_true_labels, test_pred_labels


In [ ]:
main_4()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Epoch 1:


Training:   2%|▌                               | 1/54 [01:27<1:17:27, 87.69s/it]

# Model with reference and labels

In [5]:
# test?
# Retrain the 03_model_with_ref_and_labels with the original model

# Data processing 
import pandas as pd
import numpy as np
import argparse
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling 
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# new
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import XLMRobertaModel

from transformers import BertModel, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

def main_5():
    """
    ==========================
    Parameters and Directories 
    ==========================
    """
    # Argparse parameters 
    model_name = 'bert-base-uncased'
    num_epochs = 1

    # Model parameters 
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    batch_size = 16    # Increase to 32 if using on GPU for faster training
    num_ref_labels = 12
    num_labels = 12

    # Directories 
    #input_dir = "./data/"
    output_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_with_reference_text_and_labels/"
    checkpoint_dir = "/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/output/model_with_reference_text_and_labels/checkpoints/" # path for checkpoints to be saved
    
    """
    =======
    Classes
    =======
    """
    # -- Class for creating dataset with referemce text and labels -- 
    class TextDatasetWithRefLabels(Dataset):
        def __init__(self, ref_texts, ref_labels, texts, labels, tokenizer, max_length):
            self.ref_texts = ref_texts
            self.ref_labels = ref_labels
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length

            # Binarize the reference labels
            self.mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])
            self.ref_labels = self.mlb.fit_transform(ref_labels)

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, index):
            ref_text = self.ref_texts[index]
            ref_label = self.ref_labels[index]
            text = self.texts[index]
            label = self.labels[index]

            # Tokenize the reply tweet 
            rep_tokens = self.tokenizer.tokenize(text)

            # Determine the maximum length of the reference text that can be used 
            max_ref_length = 511 - len(rep_tokens)

            # Tokenize the reference text and truncate if necessary
            ref_tokens = self.tokenizer.tokenize(ref_text)
            if len(ref_tokens) > max_ref_length:
                truncated_ref_tokens = ref_tokens[-max_ref_length:]
            else:
                truncated_ref_tokens = ref_tokens
                
            # Combine the reference and reply text tokens with a [SEP] token
            combined_tokens = truncated_ref_tokens + [self.tokenizer.sep_token] + rep_tokens 

            # Convert the combined tokens to a text string
            combined_text = self.tokenizer.convert_tokens_to_string(combined_tokens)

            # Encode the combined text
            encoded_data = self.tokenizer.encode_plus(
                combined_text,
                add_special_tokens=True,
                max_length=512,
                padding='max_length',
                truncation='only_first',
                return_attention_mask=True,
                return_tensors='pt'
            )

            input_ids = encoded_data['input_ids']
            attention_mask = encoded_data['attention_mask']

            # Convert the label data to tensors
            label_tensor = torch.tensor(label, dtype=torch.float32)
            ref_label_tensor = torch.tensor(ref_label, dtype=torch.float32)

            return {
                'input_ids': input_ids.squeeze(0),
                'attention_mask': attention_mask.squeeze(0),
                'ref_labels': ref_label_tensor,
                'labels': label_tensor
            }
    
    # -- class for modelling with reference text and labels -- 
    class BertForContextualClassification(nn.Module):
        def __init__(self, num_labels, num_ref_labels, model_name):
            super(BertForContextualClassification, self).__init__()
            self.bert = BertModel.from_pretrained(model_name)
            self.dropout = nn.Dropout(0.1)
            self.linear_ref_labels = nn.Linear(num_ref_labels, num_ref_labels) # Additional layer for reference labels
            self.linear_combined = nn.Linear(self.bert.config.hidden_size + num_ref_labels, num_labels) # Combining BERT output and reference labels

        def forward(self, input_ids, attention_mask, ref_labels):
            # Obtain BERT's output for the combined text
            output = self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            pooled_output = output[1]
            pooled_output = self.dropout(pooled_output)

            # Pass the reference labels through a linear layer
            ref_labels_output = self.linear_ref_labels(ref_labels)

            # Concatenate the BERT pooled output with the processed reference labels
            combined_output = torch.cat((pooled_output, ref_labels_output), dim=1)

            # Pass the combined output through the final linear layer
            logits = self.linear_combined(combined_output)

            return logits

    """
    =============
    Preprocessing
    =============
    """
    # -- Load Data -- 
    train_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/train/train_data_ref.csv").head(864) #smaller sample for testing
    test_data = pd.read_csv("/Users/idahelenedencker/Desktop/STANDBY_Ida/Retraining and finetuning counterspeech classifier/Data/test/test_data_ref.csv").head(216)

    #Define the labels to be binarised and transform
    mlb = MultiLabelBinarizer(classes=['Antagonizing', 'Other political statement', 'Unclassifiable', 'Hateful', 'Not Hateful', 'Unclassifiable Hateful', 'Constructive', 'Not Constructive','Unclassifiable Constructive', 'Agree', 'Disagree', 'Unclear'])
    train_labels = mlb.fit_transform(train_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)
    test_labels = mlb.transform(test_data[['rep_type', 'rep_hateful', 'rep_constructive', 'rep_agree']].values)

    # create datasets
    train_dataset = TextDatasetWithRefLabels(ref_texts=train_data['ref_text'].tolist(),
                                          ref_labels=train_data[['ref_type', 'ref_hateful', 'ref_constructive']].values,
                                          texts=train_data['rep_text'].tolist(),
                                          labels=train_labels,
                                          tokenizer=tokenizer,
                                          max_length=512)
    
    test_dataset = TextDatasetWithRefLabels(ref_texts=test_data['ref_text'].tolist(),
                                         ref_labels=test_data[['ref_type', 'ref_hateful', 'ref_constructive']].values,
                                         texts=test_data['rep_text'].tolist(),
                                         labels=test_labels,
                                         tokenizer=tokenizer,
                                         max_length=512)

    # create train and test dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    """
    =====
    Model
    =====
    """
    # Initialize the model
    model = BertForContextualClassification(num_labels, num_ref_labels, model_name)

    # Define your optimizer, loss function and learning rate
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    criterion = nn.BCEWithLogitsLoss()

    # Create training function 
    def train(model, dataloader, criterion, optimizer, device):
        model.train()
        model.to(device)
        total_loss = 0

        for batch in tqdm(dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            ref_labels = batch["ref_labels"].to(device) # Extract reference labels from the batch

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask=attention_mask, ref_labels=ref_labels) # Include reference labels when calling the model
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    # -- create evaluation function -- 
    def evaluate(model, dataloader, criterion, device):
        model.eval()
        model.to(device)
        total_loss = 0
        all_preds = []
        all_labels = []

        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            ref_labels = batch["ref_labels"].to(device) # Extract reference labels from the batch

            logits = model(input_ids, attention_mask=attention_mask, ref_labels=ref_labels) # Include reference labels when calling the model

            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = (logits.sigmoid() > 0.5).cpu().detach().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().detach().numpy())

        avg_loss = total_loss / len(dataloader)

        return avg_loss, np.array(all_labels), np.array(all_preds)
    
    # Call device 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # -- Wrap it up into a function -- 
    epoch_no = 0 

    # Define the model checkpoint file format (e.g., "model_epoch_{epoch}.pt")
    checkpoint_filename = "model_epoch_{epoch}.pt"

    # Define the label classes
    label_classes = {
        'rep_type': ['Antagonizing', 'Other political statement', 'Unclassifiable'],
        'rep_constructive': ['Constructive', 'Not Constructive', 'Unclassifiable Constructive'],
        'rep_hateful': ['Hateful', 'Not Hateful', 'Unclassifiable Hateful'],
        'rep_agree': ['Agree', 'Disagree', 'Unclear']
    }

    for epoch in range(num_epochs):
        # Free up memory by deleting variables
        epoch_no = epoch_no + 1
        print(f"Epoch {epoch_no}:")
        # -- Train model -- 
        train_loss = train(model, train_dataloader, criterion, optimizer, device)
        print(f"Train Loss: {train_loss}")

        # Save model checkpoint
        checkpoint_path = checkpoint_dir + checkpoint_filename.format(epoch=epoch_no)
        torch.save(model.state_dict(), checkpoint_path)

        # -- Test model -- 
        test_loss, test_true_labels, test_pred_labels = evaluate(model, test_dataloader, criterion, device)
        print(f"Test Loss: {test_loss}")

        # Convert output from one hot encoding 
        true_labels = np.array(test_true_labels)
        pred_labels = np.array(test_pred_labels)

        # Generate confusion matrix
        for class_name, labels in label_classes.items():
            # Get indices corresponding to labels in this class
            indices = [i for i, label in enumerate(mlb.classes_) if label in labels]
            
            # Extract true and predicted values for this class
            class_true = np.argmax(true_labels[:, indices], axis=1)
            class_pred = np.argmax(pred_labels[:, indices], axis=1)
            
            # Compute the confusion matrix
            matrix = confusion_matrix(class_true, class_pred)

            # Save or visualize the confusion matrix
            plt.figure(figsize=(10,10))
            sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels,
                    yticklabels=labels)
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.title(f'Confusion Matrix for {class_name}')
            plt.savefig(f'{output_dir}confusion_matrix_{class_name}.png')
            plt.close()

        # Generate report 
        report = classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_, output_dict=True)
        report_name = output_dir + "classification_report_" + str(epoch_no) + ".txt"

        with open(report_name, "w") as f:
            f.write(classification_report(test_true_labels, test_pred_labels, target_names=mlb.classes_))
        
        # Print results 
        #print(f"Epoch {epoch+1}:")
        print(f"  Train Loss = {train_loss:.4f}")
        print(f"  Test Loss = {test_loss:.4f}")

In [6]:
main_5()



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will b

Epoch 1:


Training:   4%|█▎                                | 2/54 [00:46<20:01, 23.10s/it]


KeyboardInterrupt: 

In [ ]:
# Retrain the 03_model_with_ref_and_labels with a new model (XLM-Roberta)